In [2]:
from pprintpp import pprint
from numpy import percentile
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
from nltk.stem.porter import *
import string
import tensorflow as tf
import random
import nltk
import pandas as pd
from fastFM import als
import scipy

2023-12-04 06:54:41.550054: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 06:54:42.722537: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-04 06:54:42.722624: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-04 06:54:42.722632: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
# Read all the data from the two data files
reviews_file = open("reviews.txt", "r")
edges_file = open("edges.txt", "r")
reviews = {}
edges = set()

while True:
    reviews_content = reviews_file.readline()
    if not reviews_content:
        break
    exec(reviews_content)
    
while True:
    edges_content = edges_file.readline()
    if not edges_content:
        break
    edge = tuple(edges_content.split())
    edges.add(edge)

reviews_file.close()
edges_file.close()

In [3]:
# Important Data Exploration Features for Reviews
    # Number of reviews
    # Number of users
    # Number of books
    # Average of stars
    # Number of each stars
    # Average of nhelpful
    # Number of missing
    # Five number summary of stars (min, max, median, 1st quartile, 3rd quartile)
    # Average number of reviews per individual
    # Number of individuals with no reviews
    # Average length of comment  
    # Flag values
    # Number of not_a_review reports
    # Number of abuse reports

# Important Data Exploration Features for Edges
    # Number of connections
    # Average number of friends
    # Number of individuals with friends
    # Number of individuals with no friends

In [4]:
## Review Statistics Processing ##
stars = []
nhelpfuls = []
comments = []
star_vals = defaultdict(int)
users_toBooks = defaultdict(set)
books_toUsers = defaultdict(set)
missing_dict = {
    'comment': 0,
    'flags': 0,
    'nhelpful': 0,
    'stars': 0,
    'time': 0,
    'unixtime': 0,
    'user': 0,
    'work': 0
}
flag_values = set()
not_a_review_count = 0
abuse_count = 0

for key, review in reviews.items():
    book_id, username = key
    users_toBooks[username].add(book_id)
    books_toUsers[book_id].add(username)

    if "stars" in review:
        stars.append(review["stars"])
        star_vals[review["stars"]] += 1
    else:
        missing_dict["stars"] += 1

    if "nhelpful" in review:
        nhelpfuls.append(review["nhelpful"])
    else:
        missing_dict["nhelpful"] += 1

    if "comment" in review:
        comments.append(review["comment"])
    else:
       missing_dict["comment"] += 1

    if "flags" in review:
        if len(review["flags"]) > 0:
            flag_values.update(review["flags"])
            if "not_a_review" in review["flags"]:
                not_a_review_count += 1
            if "abuse" in review["flags"]:
                abuse_count += 1

    missing_dict["flags"] += 1 if "flags" not in review else 0
    missing_dict["time"] += 1 if "time" not in review else 0
    missing_dict["unixtime"] += 1 if "unixtime" not in review else 0
    missing_dict["user"] += 1 if "user" not in review else 0
    missing_dict["work"] += 1 if "work" not in review else 0

review_len = len(reviews)
user_len = len(users_toBooks)
book_len = len(books_toUsers)
star_avg = sum(stars) / len(stars)
help_avg = sum(nhelpfuls) / len(nhelpfuls)
stars_min = min(stars)
stars_max = max(stars)
stars_quarts = percentile(stars, [25, 50, 75])

avg_books = 0
for user, books in users_toBooks.items():
    avg_books += len(books)
avg_books /= user_len

avg_users = 0
for book, users in books_toUsers.items():
    avg_users += len(users)
avg_users /= book_len

avg_comment = 0
for comment in comments:
    avg_comment += len(comment)
avg_comment /= len(comments)

str_flags = ", ".join(flag_values)

In [5]:
## Review Statistics Outputting ##
# Number of reviews
print("Number of reviews:", review_len)

# Number of users
print("Number of users:", user_len)

# Number of books
print("Number of books:", book_len)

# Average of stars
print("Average of stars:", star_avg)

# Number of each stars
print("Number of each stars:")
pprint(dict(star_vals))

# Average of nhelpful
print("\nAverage of nhelpful:", help_avg)

# Number of missing data points
print("Number of missing data points:")
pprint(dict(missing_dict))

# Five number summary of stars (min, max, median, 1st quartile, 3rd quartile)
print("\nMin star value:", stars_min)
print("Q1 star value:", stars_quarts[0])
print("Median star value:", stars_quarts[1])
print("Q3 star value:", stars_quarts[2])
print("Max star value:", stars_max)

# Average number of books per user
print("\nAverage number of books per user:", avg_books)

# Average number of users per book
print("Average number of users per book:", avg_users)

# Average length of comment
print("Average length of comment:", avg_comment)

# Flag values
print("\nFlag values:", str_flags)

# Number of not_a_review reports
print("Number of not_a_review reports:", not_a_review_count)

# Number of abuse reports
print("Number of abuse reports:", abuse_count)

Number of reviews: 1707070
Number of users: 83195
Number of books: 506165
Average of stars: 3.8054164873497793
Number of each stars:
{
    0.5: 7151,
    1.0: 28721,
    1.5: 9571,
    2.0: 79025,
    2.5: 37063,
    3.0: 244615,
    3.5: 116782,
    4.0: 439640,
    4.5: 85599,
    5.0: 339042,
}

Average of nhelpful: 0.17973721054203987
Number of missing data points:
{
    'comment': 0,
    'flags': 0,
    'nhelpful': 0,
    'stars': 319861,
    'time': 0,
    'unixtime': 0,
    'user': 0,
    'work': 0,
}

Min star value: 0.5
Q1 star value: 3.0
Median star value: 4.0
Q3 star value: 4.5
Max star value: 5.0

Average number of books per user: 20.51890137628463
Average number of users per book: 3.3725563798366145
Average length of comment: 793.5943446958825

Flag values: not_a_review, abuse
Number of not_a_review reports: 62395
Number of abuse reports: 13472


In [6]:
## Edges Statistics Processing ##
edge_list = defaultdict(set)

for edge in edges:
    node1, node2 = edge
    edge_list[node1].add(node2)
    edge_list[node2].add(node1)

edges_len = len(edges)

avg_friends = 0
for key, value in edge_list.items():
    avg_friends += len(value)
avg_friends /= len(edge_list)

with_friends = len(edge_list)
without_friends = user_len - with_friends

In [7]:
## Edges Statistics 
# Number of connetions
print("Number of connections:", edges_len)

# Average number of friends (sparsity of graph)
print("Average number of friends:", avg_friends)

# Number of individuals with friends
print("Number of individuals with friends:", with_friends)

# Number of individuals with no friends
print("Number of individuals with no friends:", without_friends)

Number of connections: 202178
Average number of friends: 4.840111606465896
Number of individuals with friends: 65946
Number of individuals with no friends: 17249


In [4]:
allReviews = []
for key, value in reviews.items():
    if "stars" in value and len(value["flags"]) == 0:
        allReviews.append((key[1], key[0], value))

train_set, test_set = train_test_split(allReviews, shuffle=False)

In [5]:
test_stars = []
for review in test_set:
    test_stars.append(review[2]["stars"])

def MSE(ypred):
    return np.square(np.subtract(test_stars, ypred)).mean()

In [6]:
def trunc_pred(pred):
    truncated_pred = []
    for i in range(len(pred)):
        if pred[i] < 0.5:
            truncated_pred.append(0.5)
        elif pred[i] > 5:
            truncated_pred.append(5)
        else:
            truncated_pred.append(pred[i])
            
    return truncated_pred

In [11]:
# Baseline always predicting total star average
baseline_pred = [star_avg] * len(test_set)
print("Baseline MSE:", MSE(baseline_pred))

Baseline MSE: 0.9967558349790553


In [12]:
# Simple linear regression based on length of comment and nhelpful
X = [[1, len(d[2]["comment"]), d[2]["nhelpful"]] for d in train_set]
y = [d[2]["stars"] for d in train_set]

model = linear_model.LinearRegression(fit_intercept=False)
model.fit(X, y)

X_pred = [[1, len(d[2]["comment"]), d[2]["nhelpful"]] for d in test_set]
lin_len_pred = model.predict(X_pred)
lin_len_pred = trunc_pred(lin_len_pred)

print("Linear Regression using comment length and nhelpful MSE:", MSE(lin_len_pred))

Linear Regression using comment length and nhelpful MSE: 0.9953815433412995


In [15]:
# Linear regression based on bag of words
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in train_set:
  r = ''.join([c for c in d[2]["comment"].lower() if not c in punctuation])
  for w in r.split():
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def word_feature(d):
    feat = [0] * len(words)
    r = ''.join([c for c in d['comment'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1)

    return feat

X = [word_feature(d[2]) for d in train_set]
y = [d[2]['stars'] for d in train_set]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)

X_test = [word_feature(d[2]) for d in test_set]
bow_pred = clf.predict(X_test)
bow_pred = trunc_pred(bow_pred)

print("Linear Regression using bag-of-words MSE:", MSE(bow_pred))

In [13]:
# Jaccard similarity prediction model
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)
reviewsPerUser = defaultdict(list)
reviewsPerBook = defaultdict(list)
itemAverages = {}
ratingDict = {}

for d in train_set:
    user, item = d[0], d[1]
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    reviewsPerUser[user].append(d[2])
    reviewsPerBook[book].append(d[2])
    ratingDict[(user, item)] = d[2]["stars"]
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)
    
def predictRating(user, item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['work']
        if i2 == item: continue
        ratings.append(d['stars'] - star_avg)
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x * y) for x, y in zip(ratings, similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        if item in itemAverages:
            return itemAverages[item]
        else:
            return star_avg
        
sim_pred = [predictRating(d[0], d[1]) for d in test_set]
sim_pred = trunc_pred(sim_pred)

print("Jaccard similarity prediction MSE:", MSE(sim_pred))

Jaccard similarity prediction MSE: 1.0098126449443923


In [14]:
userIDs = {}
itemIDs = {}
interactions = []

for u, i, r in allReviews:
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u, i, r['stars']))

In [15]:
# Latent factor model
optimizer = tf.keras.optimizers.legacy.Adam(0.01)
mse = tf.keras.losses.MeanSquaredError()

class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, rbu, rbi, rgu, rgi):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)], stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)], stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs), K], stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs), K], stddev=0.001))
        self.rbu = rbu
        self.rbi = rbi
        self.rgu = rgu
        self.rgi = rgi

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] + tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.rbu * tf.reduce_sum(self.betaU**2) + self.rbi * tf.reduce_sum(self.betaI**2) + self.rgu * tf.reduce_sum(self.gammaU**2) + self.rgi * tf.reduce_sum(self.gammaI**2)
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i + tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return mse(r, pred)
    
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model.call(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)

for u, i, r in interactions:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

modelLFM = LatentFactorModel(star_avg, 3, 0.00001, 0.00001, 0.00001, 0.00001)

for i in range(500):
    obj = trainingStep(modelLFM, interactions)
    #if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

lfm_pred = [modelLFM.predict(userIDs[d[0]], itemIDs[d[1]]).numpy() for d in test_set]
print("Latent factor model prediction MSE:", MSE(lfm_pred))

2023-12-04 05:33:34.314184: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-12-04 05:33:34.437606: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2023-12-04 05:33:34.437636: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-12-04 05:33:34.438337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neur

Latent factor model prediction MSE: 0.6323240356958874


In [137]:
# Sentiment analysis using negative and positive word lists
keys = []
text = []
for review in allReviews:
    user = review[0]
    item = review[1]
    comment = review[2]["comment"]

    keys.append((user, item))
    text.append(comment)

data = {'keys': keys, 'preprocess_txt': text}
df = pd.DataFrame(data)
df['total_len'] = df['preprocess_txt'].map(lambda x: len(x))

pos_words = set()
for l in open("positive-words.txt"):
    if l[0] == ";": continue
    pos_words.add(l)

neg_words = set()
for l in open("negative-words.txt"):
    if l[0] == ";": continue
    neg_words.add(l)

num_pos = df['preprocess_txt'].map(lambda x: len([i for i in x if i in pos_words]))
df['pos_count'] = num_pos

num_neg = df['preprocess_txt'].map(lambda x: len([i for i in x if i in neg_words]))
df['neg_count'] = num_neg

df['sentiment'] = round((df['pos_count'] - df['neg_count']) / df['total_len'], 3)

df = df.set_index('keys')
df.to_csv('sentiment.csv')

In [16]:
# Sentiment analysis Using NLTK
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('vader_lexicon', quiet=True)
sia = SentimentIntensityAnalyzer()

user_ids = []
book_ids = []
comments = []
neg = []
neu = []
pos = []
comp = []

for review in allReviews:
    user = review[0]
    item = review[1]
    comment = review[2]["comment"]

    score = sia.polarity_scores(comment)

    user_ids.append(user)
    book_ids.append(item)
    comments.append(comment)

    neg.append(score["neg"])
    neu.append(score["neu"])
    pos.append(score["pos"])
    comp.append(score["compound"])

In [17]:
sentiment_dict = {}
for i in range(len(user_ids)):
    sentiment_dict[(user_ids[i], book_ids[i])] = comp[i]

In [147]:
# Simple linear regression based on length of comment, nhelpful, and sentiment analysis
X = [[1, len(d[2]["comment"]), d[2]["nhelpful"], sentiment_dict[(d[0], d[1])]] for d in train_set]
y = [d[2]["stars"] for d in train_set]

model = linear_model.LinearRegression(fit_intercept=False)
model.fit(X, y)

X_pred = [[1, len(d[2]["comment"]), d[2]["nhelpful"], sentiment_dict[(d[0], d[1])]] for d in test_set]
simple_sent_pred = model.predict(X_pred)
simple_sent_pred = trunc_pred(lin_len_pred)

print("Linear Regression using comment length, nhelpful, and sentiment analysis MSE:", MSE(simple_sent_pred))

Linear Regression using comment length, nhelpful, and sentiment analysis MSE: 0.9602633065284476


In [146]:
# Linear regression with the most features based on bag of words, sentiment analysis, and length of comment
def word_feature(d):
    feat = [0] * len(words)
    r = ''.join([c for c in d[2]['comment'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1

    feat.append(sentiment_dict[(d[0], d[1])])
    feat.append(len(d[2]['comment']))
    feat.append(1)

    return feat

X = [word_feature(d) for d in train_set]
y = [d[2]['stars'] for d in train_set]

clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)

X_test = [word_feature(d) for d in test_set]
lin_sent_pred = clf.predict(X_test)
lin_sent_pred = trunc_pred(bow_pred)

print("Linear Regression using bag-of-words, sentiment analysis, and length of comment MSE:", MSE(lin_sent_pred))

Linear Regression using bag-of-words, sentiment analysis, and length of comment MSE: 0.8089379034328351


In [7]:
# Factorization Machine with no features
userIDs, itemIDs = {}, {}

for d in allReviews:
    u = d[0]
    i = d[1]
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers, nItems = len(userIDs), len(itemIDs)

X = scipy.sparse.lil_matrix((len(allReviews), nUsers + nItems))

for i in range(len(allReviews)):
    user = userIDs[allReviews[i][0]]
    item = itemIDs[allReviews[i][1]]
    X[i, user] = 1
    X[i, nUsers + item] = 1

y = np.array([d[2]['stars'] for d in allReviews])
X_train, y_train = X[:len(train_set)], y[:len(train_set)]
X_test, y_test = X[len(train_set):], y[len(train_set):]

fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

simple_fm_pred = fm.predict(X_test)
simple_fm_pred = trunc_pred(simple_fm_pred)
print("Factorization Machine with no features MSE:", MSE(simple_fm_pred))

Factorization Machine with no features MSE: 1.105352281188836


In [9]:
# Factorization Machine with an additional sentiment analysis feature
userIDs, itemIDs = {}, {}

for d in allReviews:
    u = d[0]
    i = d[1]
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)

nUsers, nItems = len(userIDs), len(itemIDs)

X = scipy.sparse.lil_matrix((len(allReviews), nUsers + nItems + 1))

for i in range(len(allReviews)):
    user = userIDs[allReviews[i][0]]
    item = itemIDs[allReviews[i][1]]
    X[i, user] = 1
    X[i, nUsers + item] = 1
    X[i, nUsers + nItems] = sentiment_dict[(allReviews[i][0], allReviews[i][1])]

y = np.array([d[2]['stars'] for d in allReviews])
X_train, y_train = X[:len(train_set)], y[:len(train_set)]
X_test, y_test = X[len(train_set):], y[len(train_set):]

fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

simple_fm_pred = fm.predict(X_test)
simple_fm_pred = trunc_pred(simple_fm_pred)
print("Factorization Machine with an additional sentiment analysis feature MSE:", MSE(simple_fm_pred))

NameError: name 'sentiment_dict' is not defined